In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

## 1. Load Data

In [2]:
df=pd.read_csv("sothebys.csv")

In [3]:
df.shape

(22711, 26)

In [4]:
#convert datatime columns
df.start_date =pd.to_datetime(df.start_date)
df.end_date =pd.to_datetime(df.end_date)

In [5]:
# Check Auction length
cnt=np.where(df["end_date"]!=df["start_date"],1,0).sum()
#cnt=0
df.drop("end_date",axis=1,inplace=True)

Therefore, we'll use $start\_date$ as the time that the auction occured going forward. 

## 2. Add Features

In [6]:
# Add year,month and season for acution time
df["auc_date"]=pd.DatetimeIndex(df.start_date).normalize()
df['auc_year'] = pd.DatetimeIndex(df['start_date']).year
df['auc_month'] = pd.DatetimeIndex(df['start_date']).month
df['auc_season']=df["start_date"].apply(lambda dt: (dt.month%12 + 3)//3)

In [8]:
# Add a column to indicate if the lot is named "untitled"
words='|'.join(["INTITULADO","UNTITLED","OHNE TITEL","SANS TITRE","SENZA TITOLO"])
df["untitled?"]=np.where(df["lot_title"].str.contains(words),1,0)

In [10]:
# Normalize "nth_in_auction" column by "auction_id" into 10 tiles
df["auc_order"]= df.groupby("auction_id")["nth_in_auction"].transform(
                     lambda x: pd.qcut(x, 10, labels=range(1,11))) 

In [11]:
# Add avg_estimate
df["avg_estimate"]=(df["estimate_low"]+df["estimate_high"])/2

## 3. Convert Currency

## 4. Export to CSV

In [ ]:
df.to_csv("final_sothebys.csv")